In [11]:
from decoder import Decoder
from decoderattn import Attention_Decoder
from encoder import Encoder
from evaluatemodel import MySeq2SeqModelEvaluation
from train_with_batch import Seq2SeqTrainer
from trainmodel import Seq2SeqTrainer
from data_proc import DataPreparation
from data_proc import *


In [9]:
import torch
import random
from io import open
import unicodedata
import string
import re
import random
import pickle
import numpy as np
import math
from mosestokenizer import *
from collections import Counter

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [23]:
obj= DataPreparation('fra', 'eng', True)
input_lang, output_lang, pairs= obj.prepare_data(reverse=True)
#input_lang, output_lang, pairs = data_prep.prepare_data('fra', 'eng', True)
print(output_lang)
print(random.choice(pairs))

Reading lines...
Form:  ['l or a . dollars l once ?', ' gold ?']
Read 2000 sentence pairs
Trimmed to 1837 sentence pairs
Counting words...
Counted words:
eng 6924
fra 6019


TypeError: DataPreparation.prepare_data() missing 1 required positional argument: 'lang2'

In [ ]:
hidden_size = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
attn_decoder1 = Attention_Decoder(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

#encoder1 = EncoderRNN(input_dim = input_lang.n_words,emb_dim =256,enc_hid_dim = 1000, dec_hid_dim = 1000, dropout = 0.6) 
encoder1 = Encoder(input_size = input_lang.n_words, hidden_size = hidden_size).to(device)
